# Sparkify Project Workspace
This workspace contains a tiny subset (128MB) of the full dataset available (12GB). Feel free to use this workspace to build your project, or to explore a smaller subset with Spark before deploying your cluster on the cloud. Instructions for setting up your Spark cluster is included in the last lesson of the Extracurricular Spark Course content.

You can follow the steps below to guide your data analysis and model building portion of this project.

In [ ]:
# import libraries
import pyspark
from pyspark.sql import SparkSession
from pyspark.sql import functions as F
from pyspark.ml.tuning import TrainValidationSplit
import plotly.express as px
import plotly.graph_objects as go

In [ ]:
# create a Spark session
spark = SparkSession \
    .builder \
    .appName("Churn") \
    .getOrCreate()

# Load and Clean Dataset
In this workspace, the mini-dataset file is `mini_sparkify_event_data.json`. Load and clean the dataset, checking for invalid or missing data - for example, records without userids or sessionids. 

In [ ]:
in_schema = "artist STRING, auth STRING, firstName STRING, gender STRING, itemInSession INT, lastName STRING," \
            "length DOUBLE, level STRING, location STRING, method STRING, page STRING, registration LONG," \
            "sessionId INT, song STRING, status int, ts LONG, userAgent STRING, userId STRING"

df_raw = spark.read.json('s3n://udacity-dsnd/sparkify/sparkify_event_data.json', schema=in_schema)\
    .withColumn('iuid', F.col('userId').cast('long')).drop('userId').withColumnRenamed('iuid', 'userId')
df_raw.head()
df_raw.printSchema()

# Exploratory Data Analysis
When you're working with the full dataset, perform EDA by loading a small subset of the data and doing basic manipulations within Spark. In this workspace, you are already provided a small subset of data you can explore.

### Define Churn

Once you've done some preliminary analysis, create a column `Churn` to use as the label for your model. I suggest using the `Cancellation Confirmation` events to define your churn, which happen for both paid and free users. As a bonus task, you can also look into the `Downgrade` events.


### Explore Data
Once you've defined churn, perform some exploratory data analysis to observe the behavior for users who stayed vs users who churned. You can start by exploring aggregates on these two groups of users, observing how much of a specific action they experienced per a certain time unit or number of songs played.

In [ ]:
df_explore = df_raw.sample(fraction=0.01, seed=14041977)

In [ ]:
df_explore.head()

In [ ]:
#per_uid_count = df_explore.groupby('userId').count()
#px.histogram(per_uid_count.collect(), x=1, labels={'1': 'interactions per user'})

In [ ]:
#per_page_count = df_explore.groupby('page').count().orderBy('count', ascending=False)
#per_page_count.show(200)
#px.bar(per_page_count.collect(), x=0, y=1, log_y=True)

# Feature Engineering
Once you've familiarized yourself with the data, build out the features you find promising to train your model on. To work with the full dataset, you can follow the following steps.
- Write a script to extract the necessary features from the smaller subset of data
- Ensure that your script is scalable, using the best practices discussed in Lesson 3
- Try your script on the full data set, debugging your script if necessary

If you are working in the classroom workspace, you can just extract features based on the small subset of data contained here. Be sure to transfer over this work to the larger dataset when you work on your Spark cluster.

# Modeling
Split the full dataset into train, test, and validation sets. Test out several of the machine learning methods you learned. Evaluate the accuracy of the various models, tuning parameters as necessary. Determine your winning model based on test accuracy and report results on the validation set. Since the churned users are a fairly small subset, I suggest using F1 score as the metric to optimize.

## Preparation
Import required classes, transform log data to features and split off test data. Will use 5% of userIds as test dataset. The remainder will be split up in the TrainValidationSplits for tested ML algorithms, to use 80% of the original data set (= 84.2% of the non-test data) for training and 15% for validation within the hyperparameter tuning.

In [ ]:
from model import LogCleanTransformer, MasterTransformer
from pyspark.ml.tuning import TrainValidationSplit, ParamGridBuilder
from pyspark.ml.evaluation import BinaryClassificationEvaluator
from pyspark.ml.classification importLinearSVC

In [ ]:
m_tra = MasterTransformer()
cleaner = LogCleanTransformer()
mt = m_tra.transform(cleaner.transform(df_raw))

In [ ]:
mt.head(30)
train, test = mt.randomSplit([0.90, 0.10], seed=1404)

## Linear SVC

In [ ]:
lsvc = LinearSVC(maxIter=300, regParam=1e-4)
#paramGrid_lsvc = ParamGridBuilder()\
#    .addGrid(lsvc.maxIter, [250, 300, 350])\
#    .addGrid(lsvc.regParam, [0, 1e-3, 1e-4, 1e-5])\
#    .build()
    
#tvs_lsvc = TrainValidationSplit(estimator=lsvc,
#                              evaluator=BinaryClassificationEvaluator(),
#                              estimatorParamMaps=paramGrid_lsvc,
#                              trainRatio=0.8333,
#                              seed=1404)
model_lsvc = lsvc.fit(train)


In [ ]:
pred_lsvc = model_lsvc.bestModel.transform(test).withColumn('tp', F.col('label') * F.col('prediction'))\
    .withColumn('fp', (1 - F.col('label')) * F.col('prediction'))\
    .withColumn('fn', F.col('label') * (1 - F.col('prediction')))

In [ ]:
pred_lsvc.cache()
pred_lsvc.head()

In [ ]:
tp_sum = pred_lsvc.agg(F.sum('tp').alias('tp_s')).collect()[0]['tp_s']
fp_sum = pred_lsvc.agg(F.sum('fp').alias('fp_s')).collect()[0]['fp_s']
fn_sum = pred_lsvc.agg(F.sum('fn').alias('fn_s')).collect()[0]['fn_s']
F1_rf = tp_sum / (tp_sum + 0.5 * (fp_sum + fn_sum))
print("Linear SVC regression F1 score :", F1_rf)

In [ ]:
model_lsvc.validationMetrics

In [ ]:
print(model_lsvc.bestModel.explainParams())

In [ ]:
model_lsvc.write().overwrite().save('s3n://jmk-udct-spotify-results/lsvc_model')

# Final Steps
Clean up your code, adding comments and renaming variables to make the code easier to read and maintain. Refer to the Spark Project Overview page and Data Scientist Capstone Project Rubric to make sure you are including all components of the capstone project and meet all expectations. Remember, this includes thorough documentation in a README file in a Github repository, as well as a web app or blog post.